In [1]:
import time
import lxml
import requests
import bs4
import json
import re
import selenium
from Settings import Settings
import Browser
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.support.ui import WebDriverWait

LID = 83055673
requestUrl = f"https://www.catawiki.com/en/l/{LID}"
webdriverPath = fr"C:\Users\DripTooHard\PycharmProjects\pythonProject1\edgedriver_win64\msedgedriver.exe"

In [5]:
soup = Browser.Browser.load_bs4("https://www.catawiki.com/en/c/599-gemstones/")

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><meta content="summary_large_image" name="twitter:card"/><meta content="@catawiki" name="twitter:creator"/><meta content="238226522900020" property="fb:app_id"/><meta content="https://www.catawiki.com/en/" property="og:url"/><meta content="website" property="og:type"/><meta content="en" property="og:locale"/><meta content="Catawiki" property="og:site_name"/><title>Gemstones for Sale in Online Auctions - Catawiki</title><meta content="index,follow" name="robots"/><meta content="Buy and sell Gemstones at Catawiki. Discover Gemstones auctions filled with special objects, selected by our experts." name="description"/><link href="https://www.catawiki.com/en/c/599-gemstones" hreflang="en" rel="alternate"/><link href="https://www.catawiki.com/nl/c/599-edelstenen" hreflang="nl" rel="alternate"/><link href="https://www.catawiki.com/de/c/599-edelsteine" hreflang="de" rel="alternate"/>

In [17]:
lot_cards = soup.find_all("a",{"class":"c-lot-card"})

In [36]:
print(lot_cards[4]["href"])

https://www.catawiki.com/en/l/82677037-1-pcs-pink-tourmaline-4-18-ct


In [42]:

def getLIDFromLink(URL):
    # Regular expression to extract only the digits following '/l/' and before the hyphen
    pattern = r"/l/([0-9]+)-"

    # Using findall to get all matches of the digit pattern
    matches = re.findall(pattern, URL)

    # Assuming there is only one match, return it directly
    if( matches):
        return matches[0]
    else:
        raise RuntimeError(f"No match for link: {URL}")

print(getLIDFromLink(lot_cards[4]["href"]))

82677037


In [7]:
pageNrs= soup.find_all("a",{"class":"c-pagination__page-link c-link no-underline u-color-mid-gray"})

In [43]:
closedSoup = Browser.SeleniumBrowser.getClosedAuctionSoup(LID)
spans = closedSoup.find_all("span",{"class":"u-no-wrap"})

C:\Users\DripTooHard\PycharmProjects\pythonProject1\Browser.py:145: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 145 of the file C:\Users\DripTooHard\PycharmProjects\pythonProject1\Browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = bs4.BeautifulSoup(html)


In [11]:
def maxOfTexts(htmlElements):
    currentMax = 0
    for htmlElement in htmlElements:
        integerValue = int(htmlElement.text)
        if(integerValue>currentMax):
            currentMax = integerValue
    return currentMax


In [12]:
maxOfTexts(pageNrs)

68

In [45]:
def turnDecimalNumberIntoInt(decimalNumber):
    return int("".join(re.findall("[0-9]+",decimalNumber)))

def getEstimatesFromSpan(spans):
    est1 = None
    est2 = None
    for span in spans:
        if(span.text[0] == "€"):
            if(est1 is None):
                est1 = turnDecimalNumberIntoInt(span.text[2:])
            elif(est2 is None):
                est2 = turnDecimalNumberIntoInt(span.text[2:])
            else:
                return "Two estimates it seems"

    if (est1 < est2):
        return (est1,est2)
    else:
        return "Est1 is larger than est2"


In [46]:
print(getEstimatesFromSpan(spans))

(800, 900)


In [19]:
driver = Browser.SeleniumBrowser.getEdgedriver()

driver.get(requestUrl)

Browser.SeleniumBrowser.declinceCookies(driver)

# Using the class names provided to extract the two values
# The class 'u-no-wrap' is assumed to be common for both elements based on your initial image
# Make sure to replace 'first_class_name' and 'second_class_name' with the actual class names you're interested in
expert_max_estimate = driver.find_elements(By.CLASS_NAME, 'u-m-r-xs') # Modify this class name

# Assuming that the first element found is the one you're interested in, extract its text
# You'll need to ensure that this actually matches the elements you want
expert_max_estimate = expert_max_estimate[0].text if expert_max_estimate else 'Not Found'

wait = WebDriverWait(driver,2)

# Use the wait to find the element that contains the "Expert Estimate" text and is within the same span element
expert_minimum_estimate = wait.until(EC.presence_of_element_located(
    (By.XPATH, "//span[normalize-space(text())='Expert Estimate']/span[@class='u-no-wrap']")
))

# Print the extracted expert estimate value
expert_minimum_estimate = expert_minimum_estimate.text if expert_minimum_estimate else 'Not Found'


print(f"Expert estimate: {expert_max_estimate}-{expert_minimum_estimate}")

# Close the Edge WebDriver
driver.quit()

Expert estimate: € 20,000-€ 18,000


[]